In [103]:
import pandas as pd
import numpy as np
import emoji

In [104]:
# combine the csv files 
# df contain all 5000 tweets
df1 = pd.read_csv('matched_tweets_4443.csv', usecols=['tweet_id','tweet_text','year','final_rating'], index_col='tweet_id')
df2 = pd.read_csv('unmatched_tweets_557.csv', usecols=['tweet_id','tweet_text','year','majority_rating'], index_col='tweet_id')
df = pd.concat([df1, df2], axis=0)

In [105]:
# remove the emoji and replace it with the its text

df['tweet_text'] = df['tweet_text'].apply(lambda x: emoji.demojize(x,  delimiters=(" ", " ")).replace("_", " ").replace("-", " "))

In [106]:
# combine the majority_rating and final_rating
df['merged_rating'] = df['majority_rating'].fillna(df['final_rating'])

In [107]:
# convert the file to csv format
# name: dataset.csv
df.to_csv('dataset.csv')

In [109]:
# split dataset.csv into relevant and irrelevant

# relevant
relevant_df = df.loc[df['merged_rating'] == 1, ['tweet_text', 'year']]
relevant_df.to_csv('relevant_dataset.csv')
relevant_df.reset_index(inplace=True)

# irrelevant
irrelevant_df = df.loc[df['merged_rating'] == 0, ['tweet_text', 'year']]
irrelevant_df.to_csv('irrelevant_dataset.csv')
irrelevant_df.reset_index(inplace=True)
